In [1]:
# imports
from pathlib import Path
from time import time
from math import ceil, log

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from utils import (
    read_single_problem_from_path_as_adjacency,
    check_validity_for_adjacency,
    transform_zero_one_clique_to_nodes,
    transform_zero_one_clique_to_nodes_of_base,
    build_clique_in_grah_from_subgraph,
    build_subgraph_from_triangles
)
from constants import BASE_INSTANCES_FILES, OTHER_INSTANCES_FILES, ALL_BEST_KNOWN
from heuristics import (
    descending_degree_glutonous_heuristic,
)
from meta_heuristics import (
    ExploringMetaHeuristicRunner,
    ExploringTaboosMetaHeuristicRunner,
    ExploringTaboosRestartMetaHeuristicRunner,
)

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DIR = INSTANCES_DIR / "other_instances"


In [3]:
# Simple run
instance = "C500.9.txt"

# Load instance
(
    number_of_nodes,
    number_of_edges,
    graph,
    degrees,
) = read_single_problem_from_path_as_adjacency(
    instance_path=OTHER_INSTANCES_DIR / instance
)
# Build subgraph
triangle, subgraph, subdegrees = build_subgraph_from_triangles(graph=graph, degrees=degrees)
# Build starting clique
starting_clique = descending_degree_glutonous_heuristic(graph=subgraph, degrees=subdegrees)
# Create runner
max_time = 10
max_intensification_without_improve = 5
max_exploring_iterations_without_improve = 3 * max_intensification_without_improve
max_restarts_iterations_without_improve = max_intensification_without_improve

runner = ExploringTaboosMetaHeuristicRunner(
    graph=subgraph,
    degrees=subdegrees,
    starting_clique=starting_clique,
    max_time=max_time,
    max_intensification_without_improve=max_intensification_without_improve,
    max_exploring_iterations_without_improve=max_exploring_iterations_without_improve,
    max_restarts_iterations_without_improve=max_restarts_iterations_without_improve,
)
(
    clique,
    time_best_found,
    iteration_of_best,
    number_of_iterations,
) = runner.meta_heuristic()
# Assert that clique is valid
assert check_validity_for_adjacency(subgraph, clique)
# Rebuild clique in starting graph
clique_in_starting_graph = build_clique_in_grah_from_subgraph(triangle=triangle, subgraph_clique=clique)
assert check_validity_for_adjacency(graph, clique_in_starting_graph)
# Print clique in nodes
print(transform_zero_one_clique_to_nodes(clique=clique_in_starting_graph))
print(transform_zero_one_clique_to_nodes_of_base(clique=clique_in_starting_graph))


[20, 27, 36, 47, 49, 64, 76, 96, 107, 109, 113, 117, 123, 132, 133, 146, 147, 148, 162, 166, 180, 181, 193, 228, 229, 243, 244, 267, 289, 290, 294, 304, 309, 311, 324, 325, 326, 331, 350, 366, 378, 380, 403, 404, 405, 406, 410, 431, 441, 450, 467, 487, 489]
[21, 28, 37, 48, 50, 65, 77, 97, 108, 110, 114, 118, 124, 133, 134, 147, 148, 149, 163, 167, 181, 182, 194, 229, 230, 244, 245, 268, 290, 291, 295, 305, 310, 312, 325, 326, 327, 332, 351, 367, 379, 381, 404, 405, 406, 407, 411, 432, 442, 451, 468, 488, 490]


In [5]:
#  Run with display
columns = [
    "instance",
    "start",
    "end",
    "bound",
    "gap",
    "start/end diff",
    "time of best",
    "iteration_of_best",
    "total_iterations",
]
display_dataframe = pd.DataFrame({column: [] for column in columns})
max_time = 30
max_intensification_without_improve = 5
max_exploring_iterations_without_improve = 3 * max_intensification_without_improve
max_restarts_iterations_without_improve = max_intensification_without_improve
# Running
for instance in OTHER_INSTANCES_FILES:
    print(instance)
    number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
        instance_path=OTHER_INSTANCES_DIR / instance
    )
    
    triangle, subgraph, subdegrees = build_subgraph_from_triangles(graph=graph, degrees=degrees)
    # Build starting clique
    starting_clique = descending_degree_glutonous_heuristic(graph=subgraph, degrees=subdegrees)

    
    runner = ExploringTaboosMetaHeuristicRunner(
        graph=subgraph,
        degrees=subdegrees,
        starting_clique=starting_clique,
        max_time=max_time,
        max_intensification_without_improve=max_intensification_without_improve,
        max_exploring_iterations_without_improve=max_exploring_iterations_without_improve,
        max_restarts_iterations_without_improve=max_restarts_iterations_without_improve,
        max_taboos = min(10, ceil(np.sqrt(np.sum(starting_clique))))
    )
    # For storing results
    # runner.update_store_variables(store_cliques=True, storing_file=f"{instance}_taboos_results.txt")

    (
        clique,
        time_best_found,
        iteration_of_best,
        number_of_iterations,
    ) = runner.meta_heuristic()
    # Assert that clique is valid
    assert check_validity_for_adjacency(subgraph, clique)
    # Rebuild clique in starting graph
    clique_in_starting_graph = build_clique_in_grah_from_subgraph(triangle=triangle, subgraph_clique=clique)
    assert check_validity_for_adjacency(graph, clique_in_starting_graph)

    # Adding to display
    new_row = [
        instance,
        np.sum(starting_clique),
        np.sum(clique),
        ALL_BEST_KNOWN[instance],
        "{:.1%}".format(1 - np.sum(clique) / ALL_BEST_KNOWN[instance]),
        "{:.1%}".format(
            np.sum(np.where(starting_clique == clique, 0, 1)) / len(subgraph)
        ),
        time_best_found,
        iteration_of_best,
        number_of_iterations,
    ]
    display_dataframe.loc[len(display_dataframe)] = new_row

print(display_dataframe)


brock800_4.txt
